### 1. Importing Necessary Libraries
Imports required Python libraries for numerical computations, deep learning, and probabilistic modeling:

- `numpy`, `torch`, and `numba` for numerical and tensor computations.
- `matplotlib.pyplot` for visualization.
- `sbi` (simulation-based inference) for probabilistic inference tasks. Here it is important to use `!pip install sbi==0.22.0` since newer `sbi` modules have a new syntax and will therefore not work.
- Custom modules (`src.mamba`, `src.temporal_encoders`) for specific modeling purposes.

### 2. Simulating Brownian Motion
The function `simulate_brownian_motion` generates sample paths of a Brownian motion with drift and diffusion:
- Uses an Euler-Maruyama scheme for stochastic differential equations.
- Incorporates a restoring force parameter `k`.
- Returns a tensor containing simulation results.

### 3. Brownian Motion Simulators
Several functions wrap `simulate_brownian_motion` to format the output differently for different neural network architectures:
- `Brownian_Motion_simulator_SSM`, `Brownian_Motion_simulator_Transformer`, etc.
- Each function modifies the output shape and resolution according to the respective model’s requirements.

### 4. Neural Network Models
The notebook implements various deep learning models for processing Brownian motion data:
- **1D CNN (`ConvNet1D`)**: A convolutional neural network for temporal pattern extraction.
- **Transformer (`TemporalTransformer`)**: Uses self-attention mechanisms for sequence modeling.
- **SSM (`TemporalMamba`)**: A structured state-space model leveraging Mamba layers.
- **Alternative CNN (`TemporalCNN`)**: Another CNN variant with optional attention mechanisms.

### 5. Model Instantiation
The last section initializes one of the models (SSM, CNN, Transformer, or 1D CNN). The selected model is assigned to the variable `model`.

In [ ]:
import numpy as np
import torch
import einops
import math
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from src.mamba import Mamba, MambaConfig 
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import analysis, utils
from sbi.inference import SNPE, simulate_for_sbi
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)
# import required modules
from sbi.utils.get_nn_models import posterior_nn
seed = 0 
torch.manual_seed(seed) 
from src.temporal_encoders import ResidualTemporalBlock, Residual, PreNorm, LinearAttention, Downsample1d, Conv1dBlock

In [2]:
def simulate_brownian_motion(num_steps=1000, dt = 5e-06, replica_num=1, D=10, x0=np.array([-1.5], dtype=np.float64), save_every=1, k=3):
    num_steps = int(num_steps)
    save_every = int(save_every)
    
    N = num_steps

    diffusion_coeffs = np.full(replica_num, D)
    Ax = diffusion_coeffs * dt
    Bx = np.sqrt(2 * Ax)
    
    x = np.zeros((num_steps//save_every, replica_num, 1), dtype=np.float64)
    xold = np.tile(x0, (replica_num, 1))

    for i in range(1, N):
        # forces evaluation
        Fx = -k * xold

        # Drawing noise 
        gx = np.random.standard_normal(size=(replica_num, 1))

        # integration
        xnew = xold + Ax[:, None] * Fx + Bx[:, None] * gx

        if (i % save_every) == 0:
            x[i // save_every] = xnew

        xold = xnew

    return x

In [3]:
def Brownian_Motion_simulator_SSM(params):
    params = np.array(params.cpu(), dtype=np.float64)
    x = simulate_brownian_motion(num_steps=1000, dt = 5e-06, replica_num=1, D=10**params[0], x0=np.array([-1.5], dtype=np.float64), save_every=1, k=3)
    return torch.tensor(x, dtype=torch.float32)

def Brownian_Motion_simulator_opt_CNN(params):
    params = np.array(params.cpu(), dtype=np.float64)
    x = simulate_brownian_motion(num_steps=5, dt=5e-06, replica_num=1, D=10**params[0], x0=np.array([-1.5], dtype=np.float64), save_every=1, k=3)
    return torch.tensor(x, dtype=torch.float32)

def Brownian_Motion_simulator_Transformer(params):
    params = np.array(params.cpu(), dtype=np.float64)
    x = simulate_brownian_motion(num_steps=1000, dt = 5e-06, replica_num=1, D=10**params[0], x0=np.array([-1.5], dtype=np.float64), save_every=1, k=3)
    x = x.reshape(-1)
    return torch.tensor(x, dtype=torch.float32)

def Brownian_Motion_simulator_1D_CNN(params):
    params = np.array(params.cpu(), dtype=np.float64)
    x =  simulate_brownian_motion(num_steps=1000, dt = 5e-06, D=10**params[0], x0=np.array([-1.5], dtype=np.float64), save_every=1, k=3)
    return torch.tensor(x, dtype=torch.float32).reshape(1, -1)

def Brownian_Motion_simulator_noCNN(params):
    params = np.array(params, dtype=np.float64)
    positions =  simulate_brownian_motion(num_steps=1000, dt = 5e-06, replica_num=1, D=10**params[0], x0=np.array([-1.5], dtype=np.float64), save_every=1, k=3)
    positions = positions.reshape(-1)
    return torch.tensor(positions, dtype=torch.float32)

In [5]:
#1D CNN
class ConvNet1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 100, kernel_size=3),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.MaxPool1d(20),
            )
        self.layer2 = nn.Flatten(start_dim=1)
        self.layer3 = nn.Sequential(
            nn.Linear(4900, 100),  
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Linear(100,6),
            nn.Softmax(dim=1))

    def forward(self, x):
        x=x.reshape(-1,1,1000)
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

#Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TemporalTransformer(nn.Module):
    def __init__(self, transition_dim, dim=32, kernel_sizes=(4, 4), stride=2, num_heads=4, depth=4, mlp_dim=32):
        super().__init__()

        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=transition_dim if i == 0 else dim, out_channels=dim, 
                      kernel_size=ks, stride=stride, padding=ks//2)
            for i, ks in enumerate(kernel_sizes)
        ])

        self.pos_embedding = PositionalEncoding(dim, dropout=0.1, max_len=5000)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=dim, 
            nhead=num_heads, 
            dim_feedforward=mlp_dim,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=depth)

        self.to_out = nn.Linear(dim, 1)

    def forward(self, x):
        '''
            x : [ batch x horizon x transition ]
        '''
        
        x=x.unsqueeze(2)
        
        x = einops.rearrange(x, 'b h t -> b t h')

        for conv in self.conv_layers:
            x = F.relu(conv(x))

        x = einops.rearrange(x, 'b t h -> b h t')
        x = self.pos_embedding(x)

        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1) 
        x = torch.cat((cls_tokens, x), dim=1)  # Shape: [batch, horizon + 1, dim]
        
        x = self.transformer(x)  

        x = x[:, 0]
        x = self.to_out(x)
        return x

#Optional Attention
class TemporalCNN(nn.Module):
    def __init__(
        self,
        transition_dim,
        dim=32,
        dim_mults=(1, 2, 4),
        attention=True,
        padding_mode='reflect',
        kernel_size=3,
    ):
        super().__init__()

        dims = [transition_dim, *map(lambda m: dim * m, dim_mults)]
        in_out = list(zip(dims[:-1], dims[1:]))
        print(f'[ models/temporal ] Channel dimensions: {dims}')

        self.downs = nn.ModuleList([])
        self.ups = nn.ModuleList([])
        num_resolutions = len(in_out)

        for ind, (dim_in, dim_out) in enumerate(in_out):
            is_last = ind >= (num_resolutions - 1)

            self.downs.append(nn.ModuleList([
                ResidualTemporalBlock(dim_in, dim_out, kernel_size = kernel_size, padding_mode=padding_mode),
                ResidualTemporalBlock(dim_out, dim_out, kernel_size = kernel_size, padding_mode=padding_mode),
                Residual(PreNorm(dim_out, LinearAttention(dim_out))) if attention else nn.Identity(),
                Downsample1d(dim_out) if not is_last else nn.Identity()
            ]))

        mid_dim = dims[-1]
        self.mid_block1 = ResidualTemporalBlock(mid_dim, mid_dim, kernel_size = kernel_size, padding_mode=padding_mode)
        self.mid_attn = Residual(PreNorm(mid_dim, LinearAttention(mid_dim))) if attention else nn.Identity()
        self.mid_block2 = ResidualTemporalBlock(mid_dim, mid_dim, kernel_size = kernel_size, padding_mode=padding_mode)

        self.proj_out = nn.Linear(mid_dim, 1)

    def forward(self, x):
        '''
            x : [ batch x horizon x transition ]
        '''

        #reshape to [batch x transition x horizon]

        x = x.squeeze(-1)
        
        x = einops.rearrange(x, 'b h t -> b t h')
     
        
        for resnet, resnet2, attn, downsample in self.downs:
            x = resnet(x)
            x = resnet2(x)
            x = attn(x)
            x = downsample(x)

        x = self.mid_block1(x)
        x = self.mid_attn(x)
        x = self.mid_block2(x)
        x = x.mean(dim=-1)
        x = self.proj_out(x)
        return x

#SSM
class TemporalMamba(nn.Module):
    def __init__(
        self,
        transition_dim,
        dim=128,
        kernel_size=3,
        expand=2,
        num_layers=1, 
    ):
        super().__init__()

        self.mamba_layers = nn.ModuleList() #layer mamba
        self.l_norm_layers = nn.ModuleList() #layer norm

        for _ in range(num_layers):
            config = MambaConfig(n_layers=num_layers, d_model=dim, d_state=dim, d_conv=kernel_size, expand_factor=expand)
            self.mamba_layers.append(Mamba(config))
        
            self.l_norm_layers.append(nn.LayerNorm(dim))
        
        self.x_emb = nn.Linear(transition_dim, dim) #layer embedding
        self.proj_out = nn.Linear(dim, 1) #layer output

    def forward(self, x):
        '''
            x : [ batch x horizon x transition ]
        '''
        x = x.squeeze(-1)
        x = self.x_emb(x)  
        for mamba, l_norm in zip(self.mamba_layers, self.l_norm_layers):
            x_in = x
            x = mamba(x)
            x = l_norm(x + x_in)
        x = x[:, -1]
        x = self.proj_out(x)
        return x

In [ ]:
model = TemporalMamba(transition_dim=1, dim=32, kernel_size=3, expand=2, num_layers=2) #SSM
#model = TemporalCNN(1, dim=32, dim_mults=(1, 2, 4), attention=True, padding_mode='reflect', kernel_size=1) #Opt CNN
#model = TemporalTransformer(transition_dim=1, dim=32, kernel_sizes=(4, 4), stride=2, num_heads=4, depth=4, mlp_dim=32) # Transformer 
#model = ConvNet1D() #1DCNN

In [6]:
low_limit = [-1]
high_limit = [2]

In [ ]:
prior = utils.BoxUniform(
    low = torch.tensor([low_limit[0]], device='cuda'),
    high = torch.tensor([high_limit[0]], device='cuda')
)


prior, num_parameters, prior_returns_numpy= process_prior(prior)

simulator_wrapper = process_simulator(Brownian_Motion_simulator_SSM, prior, prior_returns_numpy)

check_sbi_inputs(simulator_wrapper, prior)

In [8]:
Brownian_Motion_simulator, prior = prepare_for_sbi(Brownian_Motion_simulator_SSM, prior)

In [9]:
neural_posterior = posterior_nn(model='nsf', embedding_net = model)

inference = SNPE(prior, device = 'cuda', density_estimator=neural_posterior)

In [ ]:
# run the inference procedure on one round and 10000 simulated data points
theta, x = simulate_for_sbi(simulator_wrapper, prior, num_simulations=50000, num_workers=1)

In [ ]:
density_estimator = inference.append_simulations(theta, x, data_device='cuda').train(training_batch_size=128, show_train_summary=True)

posterior = inference.build_posterior(density_estimator)

In [ ]:
with open('your_path.pkl', 'rb') as f:
    posterior = torch.save(f) #save the posterior for later use